In [1]:
# imports
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import os
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
from urllib.parse import urlparse
from googlesearch import search  # Free Google search library
# import validators

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
class Website:
    headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=self.headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
url='https://neuropathbhc.org/'
with open('jobDesc.txt','r') as f:
    job_description=f.readlines()
# job_description="""    About Etched

# Etched is building AI chips that are hard-coded for individual model architectures. Our first product (Sohu) only supports transformers, but has an order of magnitude more throughput and lower latency than a B200. With Etched ASICs, you can build products that would be impossible with GPUs, like real-time video generation models and extremely deep chain-of-thought reasoning.

# Backed by $150M+ in funding from investors like Peter Thiel, Jane Street, and Two Sigma, we're scaling rapidly and looking for exceptional interns to join us in Summer 2025. Interns will get to work on cutting-edge engineering and business challenges across the following departments at Etched:

# Hardware Engineering (ASIC):

# Our Hardware team is designing next-generation silicon that runs transformer models faster than any chip in history. Led by Ajat Hukkoo (ex-Broadcom Distinguished Engineer, Intel VP) and Mark Ross (former Cypress CTO; shipped 5 different $1B+ silicon systems), our Hardware team is looking for excellent electrical and computer engineers to help us design and verify our next-gen silicon.

# Architecture:

# Our Architecture team, led by Saptadeep Pal (Co-founder/Chief Architect at Auradine, ex-NVIDIA architecture team), is designing the underlying chip architecture for our silicon. Architecture interns will help solve novel challenges in microarchitecture design to drive unprecedented FLOPS on our chip.

# Platform Engineering:

# Our Platform team is led by Brian Loiler, NVIDIA's former Senior Director of Datacenter Systems Engineering who built multiple generations of DGX and HGX. Scaling from single chips to full datacenter deployments requires solving complex challenges in thermal design, power delivery, system integration, and more. We're looking for exceptional electrical and mechanical engineers to join the Platform team and help us build state-of-the-art systems at scale.

# Software Engineering and Machine Learning:

# Our Software and Machine Learning teams build firmware, inference engines, and model infrastructure for Sohu. Our stack is engineered to optimize transformer performance - enabling capabilities that are impossible on general-purpose chips. The team is led by David Munday, who helped lead data center and TPU infrastructure at Google and DeepMind for 7 years. We're looking for leading SWEs and machine learning researchers across a variety of roles.

# Operations:

# The operations team is our special forces unit, working on the company's most critical business initiatives alongside our COO, Robert Wachen. We tackle problems through rigorous analysis, strategic planning, and relentless execution. We're looking for sharp, ambitious interns who can thrive under pressure, take complete ownership of complex problems, and deliver measurable results.

# Go-to-Market:

# The Go-to-Market team crafts the strategy and executes the plans to bring Sohu to market, helping customers leverage Sohu to build previously-impossible products. The team includes Chase Holmes, former top-seller at Databricks and MosaicML, and Alexandra Boyle, former CCO at Symphony, and operates under Robert, our COO who previously scaled Prod, an AI accelerator, to a $7B+ cohort valuation. We're looking for talented customer-facing interns to help accelerate our go-to-market engine.

# Talent Acquisition:

# The Talent team identifies and attracts the people who make up Etched. The team works directly with leadership to assemble world-class experts across all departments. We're looking for technical recruiting interns who can build relationships and source top talent.

# Program Details:

# 12-week paid internship (June - August 2025)

# Generous housing support for those relocating

# Based at our office in San Jose, CA

# Direct mentorship from industry leaders and world-class engineers

# Opportunity to work on one of the most important problems of our time

# For any questions, contact internships@etched.com."""

In [8]:
class SummarizationAgent:
    def __init__(self):
        self.link_system_prompt = "You are provided with a list of links found on a webpage. \
        You are able to decide which of the links would be most relevant to include in a brochure about the company, \
        such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
        
        self.link_system_prompt += "You should respond in JSON as in this example:"
        self.link_system_prompt += """
        {
            "links": [
                {"type": "about page", "url": "https://full.url/goes/here/about"},
                {"type": "careers page": "url": "https://another.full.url/careers"}
            ]
        }
        """

        self.link_system_prompt = "You are provided with a list of links found on a webpage. \
        Your task is to determine which links are most relevant for finding job-related information, \
        including job postings, team structures, related projects, departments, company research, and company news. \
        Prioritize pages that provide context about the job role and its industry impact.\n"
        self.link_system_prompt += "You should respond in JSON as in this example:"
        self.link_system_prompt += """
                {
                    "links": [
                        {"type": "job posting", "url": "https://full.url/goes/here/job-title"},
                        {"type": "careers page", "url": "https://full.url/goes/here/careers"},
                        {"type": "team/department page", "url": "https://full.url/goes/here/team"},
                        {"type": "project/research page", "url": "https://full.url/goes/here/project"},
                        {"type": "company news", "url": "https://full.url/goes/here/news"}
                    ]
                }
        """

        self.system_prompt = "You are an assistant that searches for and extracts all available information related to a job posting from a company website. \
                Your goal is to gather as much detail as possible to help the user understand the job role, its responsibilities, \
                the projects it is related to, the team or department it belongs to, and any background about the company’s work in this area. Find anything that can be related to keywords in job description\
                Analyze relevant webpages, including careers, company projects, teams, and news, to provide a comprehensive understanding \
                of the job in its full context. Preserve all relevant text and descriptions while ensuring clarity and completeness. \
                Respond in Markdown format."

        load_dotenv(override=True)
        api_key = os.getenv('OPENAI_API_KEY')

        self.MODEL = 'gpt-4o-mini'
        self.openai = OpenAI()
        

    
    def generate_search_term(self, job_description):
        """Use OpenAI to generate an optimized search term based on the job description."""
        # V2
        prompt = f"""
                    Given the following job description, generate an optimized search term to find **relevant background information** about this role. 
                    Avoid returning direct job postings or career listings that repeat the job description.
                    
                    The search term should help find:
                    - **Company projects** related to this job role.
                    - **Industry focus and research** relevant to this field.
                    - **Technological advancements, trends, or recent developments** in this domain.
                    - **Insights about the company’s work culture, leadership, and market positioning**.
                    
                    **Do NOT include keywords that return job listings or career pages**.
                    
                    ### Job Description:
                    {job_description}
                    
                    Provide ONLY the search term without explanation.
                    """

        # V1
        # prompt = f"""
        # Given the following job description, generate the best search term to find relevant information about this role, 
        # including company projects, industry focus, and related news. The search term should be concise and effective for Google search.
    
        # Job Description:
        # {job_description}
    
        # Provide only the search term without explanation.
        # """
    
        try:
            response=self.openai.chat.completions.create(
                model=self.MODEL,
                messages=[{"role": "system", "content": "You are an expert at creating effective Google search queries."},
                              {"role": "user", "content": prompt}],
            )
            # response = openai.chat.completions.create(
            #     model=MODEL,
            #     messages=[{"role": "system", "content": "You are an expert at creating effective Google search queries."},
            #               {"role": "user", "content": prompt}],
            #     # max_tokens=20
            # )
            return response.choices[0].message.content 
        except Exception as e:
            print(f"Error generating search term: {e}")
            return ""

    
    def check_url_reachable(self, url):
        try:
            response = requests.head(url, allow_redirects=True, timeout=5)  # HEAD is faster than GET
            return response.status_code == 200
        except requests.RequestException:
            return False

    def google_search(self, search_query, num_results=20):
        """Perform a Google search and get the first `num_results` relevant links (Free method)."""
        links = []
        try:
            
            for url in search(search_query, num_results=num_results):#, stop=num_results, pause=2):
                if self.check_url_reachable(url):
                    links.append(url)

        except Exception as e:
            print(f"Error during Google search: {e}")
        
        return links
        
    def get_links_user_prompt(self, website, job_description):
        user_prompt = f"Here is a job description for a position at {website.url}:\n\n{job_description}\n\n"
        user_prompt += "Please determine which pages on the company website are relevant to this job and its context, including job details (job responsibilities, Required Qualifications, preferred requirements, job title), \
                        related projects, teams, departments, research, and company news. Identify all relevant web links and return the full \
                        https URLs in JSON format. Do not include Terms of Service, Privacy Policy, email links, or unrelated pages.\n"
        user_prompt += "Links (some might be relative links):\n"
        user_prompt += "\n".join(website.links)


        
        # user_prompt = f"Here is the list of links on the website of {website.url} - "
        # user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
        #                 Do not include Terms of Service, Privacy, email links.\n"
        # user_prompt += "Links (some might be relative links):\n"
        # user_prompt += "\n".join(website.links)
        return user_prompt


    def get_links(self, url, job_desc):
        website = Website(url)
        # completion: your task is completing this conversation
        response = self.openai.chat.completions.create(
            model=self.MODEL,
            messages=[
                {"role": "system", "content": self.link_system_prompt},
                {"role": "user", "content": self.get_links_user_prompt(website, job_desc)}
          ],
            response_format={"type": "json_object"} # we tell OpenAI, we want Json object back in its response. OpenAI in its documentation recommend that it's still important that you mention in your prompt that a json response is required even if you specify format in this argument.
        )
        result = response.choices[0].message.content 
        # dot choices zero. So what's this about? Well, as it happens we can actually in the API request ask to have multiple variations if we want, 
        # if we wanted it to generate several possible variations of the response. And we haven't done that. So we're only going to get back one.
        # Uh, and so those variations come back in the form of these choices. But we've only got one. So choices zero is getting us the one and the only choice of the response back.
        return json.loads(result)  # we use json.loads to bring it back as JSON.


    # get content of page and link. Go into links and collect content
    # def get_all_details(self, url):
    #     result = "Landing page:\n"
    #     result += Website(url).get_contents()
    #     links = self.get_links(url)
    #     print("Found links:", links)
    #     for link in links["links"]:
    #         result += f"\n\n{link['type']}\n"
    #         result += Website(link["url"]).get_contents()
    #     return result

    
    # def get_all_details(self, url, job_desc, company_name, visited=None):
    #     if visited is None:
    #         visited = set()
    
    #     # Avoid revisiting the same page
    #     if url in visited:
    #         return ""
    
    #     visited.add(url)
    
    #     result = f"Page: {url}\n"
    #     result += Website(url).get_contents()
    
    #     # Get all links from the current page
    #     links = self.get_links(url, job_desc)
    
    #     for link in links.get("links", []):
    #         link_url = link["url"]
    
    #         # Ensure the link belongs to the same company domain
    #         if self.is_same_domain(url, link_url):  
    #             result += f"\n\n{link['type']}\n"
    #             result += self.get_all_details(link_url, job_desc, visited)  # Recursive call
    
    #     return result, visited
    
    
    # def is_same_domain(self, base_url, target_url):
    #     """Helper function to check if the target_url is within the same company domain."""
    #     base_domain = urlparse(base_url).netloc
    #     target_domain = urlparse(target_url).netloc
    #     return base_domain == target_domain
    
        
    def get_all_details(self, url):
        result = "Landing page:\n"
        result += Website(url).get_contents()
        return result

    def get_brochure_user_prompt(self,company_name, job_desc, url):
        # user_prompt = f"You are looking at a company called: {company_name}\n"
        # user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
        # user_prompt += self.get_all_details(url)
        user_prompt = f"You are researching a company called: {company_name}.\n\n"
        user_prompt += "Below are the contents of its landing page and other relevant pages. Your goal is to extract and analyze \
                        all information related to a specific job position to help the user understand its full context within the company.\n\n"

        
        # Google Search
        search_term = self.generate_search_term(job_description)
        search_query = f"{search_term} {company_name}"  # Append company name to the search term
    
        print(f"Searching Google for: {search_query}")
        links = self.google_search(search_query, num_results=20)
        print(links)
        for link in links:
            user_prompt += self.get_all_details(link)
            
        user_prompt += "### **Instructions:**\n"
        # user_prompt += "- Identify and extract all details about the job, including responsibilities, qualifications, and benefits.\n"
        user_prompt += "- Find information on the department, team, or projects this job is associated with.\n"
        user_prompt += "- Look for company initiatives, research, or ongoing work that may relate to this role.\n"
        user_prompt += "- Extract any relevant company culture, values, or work environment details that impact this position.\n"
        user_prompt += "- If available, include company news, innovations, or industry focus related to the job role.\n"
        user_prompt += "- Provide a structured and comprehensive response in Markdown format for clarity.\n"
        user_prompt += "- Only show the connections in job descriptions and the webpage content to see what are related things\n"
        user_prompt += "- focus on everything that is related to the job description especially the job title/position title\n"
        # user_prompt += "- if you do not find the related content, just output no information found. Do not generate unnecessary info\n"
        user_prompt += " provide as much information as you can. Also, do not include the job description in the output. Response in markdown format\n"
        user_prompt += "\n### **Job & Company Information:**\n"

        print('length of the prompt: ',len(user_prompt))
        if len(user_prompt)>10000:
            user_prompt = user_prompt[:10_000] # Truncate if more than 10,000 characters, just in case
            
        return user_prompt


    def create_brochure(self, company_name, url, job_desc):
        response = self.openai.chat.completions.create(
            model=self.MODEL,
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": self.get_brochure_user_prompt(company_name, job_desc, url)}
              ],
        )
        result = response.choices[0].message.content
        display(Markdown(result))


In [9]:
SA=SummarizationAgent()
SA.create_brochure('NeuroPath Behavioral Health',url, job_description)

Searching Google for: "AI projects machine learning developments industry trends organizational culture insights" NeuroPath Behavioral Health
['https://trendsresearch.org/insight/ai-in-mapping-neural-pathways-for-neuroscience/?srsltid=AfmBOorHxiIQgJX9QrpVcTSz98Gp_KzpXNC-d1q1y5-lupnljvF6aSUz', 'https://maximus.com/life-at-maximus', 'https://www.linkedin.com/jobs/view/it-intern-ai-machine-learning-at-neuropath-behavioral-healthcare-4049563288', 'https://alzres.biomedcentral.com/articles/10.1186/s13195-024-01540-6', 'https://link.springer.com/article/10.1007/s10462-024-10948-3', 'https://www.nature.com/articles/s41746-023-00868-x', 'https://www.boozallen.com/c/insight/thought-leadership/life-science-research.html', 'https://www.numberanalytics.com/blog/exploring-ai-trends-pharma-healthcare-futures', 'https://neuro.jmir.org/2024/1/e56665/PDF', 'https://mcml.ai/research/areac/', 'https://www.milbank.org/quarterly/opinions/leveraging-artificial-intelligence-to-bridge-the-mental-health-workfo

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


length of the prompt:  1603156


# NeuroPath Behavioral Health Job Posting Analysis

## Company Overview
**Company Name:** NeuroPath Behavioral Health  
**Specialization:** Integration of Artificial Intelligence in Neuroscience

NeuroPath Behavioral Health focuses on the intersection of neuroscience and artificial intelligence, particularly in mapping neural pathways. The company aims to revolutionize both diagnosis and treatment of neurological disorders through advanced AI techniques, including machine learning and neural networks. The ongoing research explores applications in personalized medicine and brain-computer interfaces (BCIs).

---

## Job Position Details
**Position Title:** [Job Title Not Specified in Provided Information]  
**Reported By:** Noor Al Mazrouei, Senior Researcher/ Head of the AI Program  
**Posted On:** March 14, 2025

### Key Responsibilities
While the specific job title and responsibilities are not explicitly mentioned in the provided information, the outlined duties and focus areas of the position, based on surrounding context, may include:

- **Research and Development**: Conducting research on how AI can analyze and decode brain activity patterns via methods such as machine learning and deep neural networks.
- **Data Analysis**: Utilizing AI algorithms to identify previously undetected patterns in neural pathways and genetic components to inform therapeutic strategies.
- **Collaboration**: Working within a multi-disciplinary team to bridge theoretical neuroscience concepts with practical AI applications.
- **Innovation in BCIs**: Engaging in projects that enhance communication between human cognition and machine processing through brain-computer interfaces.
- **Focus on Personalized Medicine**: Leveraging AI for advancements in personalized treatment plans for neurological disorders, enhancing the diagnostic process of complex conditions.

### Projects and Topics
The job position is contextualized within several pivotal projects that include:
- **Mapping Neural Pathways**: Investigating the biological complexities of neural pathways with AI.
- **Predictive Analytics in Neuroscience**: Enhancing understanding of cognitive functions through the prediction of cognitive states and behaviors.
- **Personalized Medicine Initiatives**: Creating innovative models for individualized treatment based on genomic and omics data.
- **BCI Innovations**: Fostering breakthroughs in brain-computer interface technology and real-time adaptive treatments.

---

## Research and Development Emphasis
The involvement in these research areas illustrates that NeuroPath Behavioral Health is not only dedicated to advancing theoretical knowledge in neuroscience but also to implementing practical solutions that can lead to significant improvements in patient care. This aligns with the overarching mission of expanding our understanding of brain functions and enhancing treatment efficacy for a range of neurological disorders.

### Expected Qualifications (Inferred)
While specific qualifications for the job position are not listed, potential candidates may be expected to possess:
- Background in neuroscience, computer science, or a related field
- Experience with machine learning frameworks and AI technologies
- Proficiency in data analysis and interpretation
- Familiarity with brain-computer interface projects and personalized medicine
- Excellent teamwork and communication skills

---

## Conclusion
In summary, the job position at NeuroPath Behavioral Health appears to focus on leveraging cutting-edge AI technologies to advance neuroscience research, with an ultimate goal of enhancing the diagnosis and treatment of neurological disorders. The dynamic role requires interdisciplinary collaboration and emphasizes innovation in personalized medicine and brain-computer interfaces. If further details on the exact job title and specific qualifications are available, they should be consulted for a complete understanding.